In [1]:
!pip install transformers==4.10.3
!pip install nlp==0.2.0
!pip install datasets
!pip install pytorch-crf==0.7.2
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 4.8 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Uninstalling transformers-4.20.1:
      Successfully uninstalled transformers-4.20.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.0 requires protobuf==3.20.0, but you have protobuf 3.19.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.1/857.1 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 356.3 kB/s eta 0:00:00a 0:00:01
  Preparing metadata 

In [2]:
import numpy as np
import torch
import torch.nn as nn
import transformers
import datasets
import nlp
import logging
from transformers import BertPreTrainedModel, BertModel
from torchcrf import CRF
import torch.nn.functional as F
logging.basicConfig(level=logging.INFO)

In [3]:
## Resembling our dataset with Squad
import json
from tqdm import tqdm
import pandas as pd
import datasets
from sklearn.model_selection import train_test_split

# Train data = quoteR data
import json
from pathlib import Path


def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    count = 1
    ids = []
    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    answer['answer_start'] = [answer['answer_start']]
                    answer['text'] = [answer['text']]
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
                    ids.append('quoter_%s'%count)
                    count += 1

    return ids, contexts, questions, answers


ids, contexts, questions, answers = read_squad('../input/quote-recommendation/quote_to_squad_train.json')


quote_dict_quoter = {
    "id": ids,
    "context": contexts,
    "question": questions,
    "answers": answers
}


ds_quoter = datasets.Dataset.from_dict(quote_dict_quoter)


with open('../input/quote-recommendation/overall_data_qa_formatted_new.json', 'rb') as content:
    context_quote_paragraph_gandhi = json.load(content)

# with open('/content/quotus_converted.json', 'rb') as test:
#     context_quote_test = json.load(test)


count = 1
ids, train_contexts, train_questions, train_answers = [], [], [], []

for context, qa in context_quote_paragraph_gandhi.items():
    contexts = []
    questions = []
    answers = []
    
    ans_dict = {'answer_start': [qa['start_index']], 'text': [str(qa['quote'])]}

    train_contexts.append(str(qa['paragraph']))
    train_questions.append(str(context))
    train_answers.append(ans_dict)
    ids.append("gandhi_%s"%count)
    count+=1 


quote_dict_gandhi = {
    "id": ids,
    "context": train_contexts,
    "question": train_questions,
    "answers": train_answers
}


ds_gandhi = datasets.Dataset.from_dict(quote_dict_gandhi)

# # 80% train, 20% test + validation
# train_testvalid = ds.train_test_split(0.2)
# # gather everyone if you want to have a single DatasetDict
# quote_dataset = datasets.DatasetDict({
#     'train': train_testvalid['train'],
#     'validation': train_testvalid['test']})

quote_dataset = datasets.DatasetDict({
    'train': ds_quoter,
    'validation': ds_gandhi})
quote_dataset


DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 10253
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 526
    })
})

In [4]:
print(quote_dataset['train'][5])

{'id': 'quoter_6', 'context': 'Eliot, Hampden, Lilburne, Milton, Shaftesbury and Locke. Could there be a better illustration of Sir Frederick Pollock\'s noble tribute to the genius of the common law:      "Remember that Our Lady, the Common Law, is not a task-mistress, but     a bountiful sovereign, whose service is freedom. The destinies of     the English-speaking world are bound up with her fortunes and     migrations and its conquests are justified by her works"?  Another reason makes the consideration of the subject not only interesting but opportune. "These are the times that try men\'s souls." It is a time of sifting, when men of all nations in civilization in these critical days are again testing the value even of those political institutions which have the sanction of the past. Society is in a state of flux. Everywhere the foundations of governmental structures seem to be settling--let us hope and pray upon a _surer_ foundation--and when the seismic convulsion of the world war

In [5]:
## Creating NER data
import json
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Features, Value, ClassLabel, Sequence
class_names = ["O", "B-QUOTE", "I-QUOTE"]



with open('../input/quote-recommendation/overall_data_qa_formatted.json', 'rb') as content:
    context_quote_paragraph = json.load(content)

# with open('/content/quotus_converted.json', 'rb') as test:
#     context_quote_test = json.load(test)

print(len(context_quote_paragraph))

count = 1

ids,  tokens, spans, ner_tags = [], [], [], []

for context, qa in context_quote_paragraph.items():
    contexts = []
    questions = []
    answers = []
    
    ans_dict = {'answer_end': [qa['end_index']], 'answer_start': [qa['start_index']], 'text': [str(qa['quote'])]}


    tokens.append(str(qa['paragraph']).split(" "))
    spans.append(['QUOTE: %s'%str(qa['quote'])])

    # ner_tags: a list of classification labels, with possible values including O (0), B-QUOTE (1), I-QUOTE (2)
    ners = [0] * len(str(qa['paragraph']).split(" "))

    ners[qa['start_index'] - 1] = 1
    for i in range(qa['start_index'], qa['end_index']):
        ners[i] = 2

    ner_tags.append(ners)
    ids.append("gandhi_%s"%count)
    count+=1 

quote_ner_dataframe = pd.DataFrame({
    "id": ids,
    "tokens": tokens,
    "ner_tags": ner_tags,
    "spans": spans
})


# quote_ner_dataframe.drop_duplicates(subset=['tokens'], keep=False)
# quote_ner_dataframe = quote_ner_dataframe.drop('__index_level_0__', axis=1)


features = Features({'id': Value('string'), 'tokens': Sequence(Value("string")),  
                     'ner_tags': Sequence(ClassLabel(names=class_names)), 'spans': Value('string')})
train, validation =train_test_split(quote_ner_dataframe, test_size=0.2)

# quote_ner_dataset = datasets.DatasetDict({'train':datasets.Dataset.from_pandas(train),  
#                                           'validation': datasets.Dataset.from_pandas(validation)})

dic = {
    "id": ids,
    "tokens": tokens,
    "ner_tags": ner_tags,
    "spans": spans
}

ds = datasets.Dataset.from_dict(dic, features)

# 80% train, 20% test + validation
train_testvalid = ds.train_test_split(0.2)
# gather everyone if you want to have a single DatasetDict
quote_ner_dataset = datasets.DatasetDict({
    'train': train_testvalid['train'],
    'validation': train_testvalid['test']})

ner_label_names = quote_ner_dataset["train"].features["ner_tags"].feature.names
quote_ner_dataset

526


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'spans'],
        num_rows: 420
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'spans'],
        num_rows: 106
    })
})

In [6]:
squad_v2 = False
dataset_dict = {
    "qa": quote_dataset,
    "ner": quote_ner_dataset,
}

In [7]:
print(dataset_dict['ner']['train'].features["ner_tags"].feature.names)

['O', 'B-QUOTE', 'I-QUOTE']


In [8]:
for task_name, dataset in dataset_dict.items():
    print(task_name)
    print(dataset_dict[task_name]["train"][0])
    print()

qa
{'id': 'quoter_1', 'context': 'bang-up modern machine shooter is no more murderous for me than yours is in your hands. \'Sufficient unto the day is the evil thereof!\' and I\'m ready to compare notes at the end of our little expedition, to see who has slaughtered the most game," and Bluff wagged his round head with its thatch of yellow hair, defiantly.  "Well, a man is known by the company he keeps, and any true sportsman--" began Jerry, ready to open the discussion on the spot.  "Rats!" exclaimed Will, as he got in readiness to mount his machine; "stow all that hot air until the first chilly night. Perhaps you\'ll need it before long. I say, Frank?"  "Well, what?"  "Has the wagon started along?" asked the other, eagerly.  "Yes, I saw it off before coming over here. Everything\'s aboard, and unless old Uncle Toby has an accident on the road, he guarantees to get up there shortly after noon," replied the leader, quietly.  "So, you got your hired man to do the driving; and I\'ve half 

## Creating a Multitask Model

In [9]:
class MultitaskModel(transformers.PreTrainedModel):
    def __init__(self, encoder, taskmodels_dict):
        """
        Setting MultitaskModel up as a PretrainedModel allows us
        to take better advantage of Trainer features
        """
        super().__init__(transformers.PretrainedConfig())

        self.encoder = encoder
        self.taskmodels_dict = nn.ModuleDict(taskmodels_dict)

    @classmethod
    def create(cls, model_name, model_type_dict, model_config_dict):
        """
        This creates a MultitaskModel using the model class and config objects
        from single-task models. 

        We do this by creating each single-task model, and having them share
        the same encoder transformer.
        """
        shared_encoder = None
        taskmodels_dict = {}
        for task_name, model_type in model_type_dict.items():
            model = model_type.from_pretrained(
                model_name, 
                config=model_config_dict[task_name],
            )
            if shared_encoder is None:
                shared_encoder = getattr(model, cls.get_encoder_attr_name(model))
            else:
                setattr(model, cls.get_encoder_attr_name(model), shared_encoder)
            taskmodels_dict[task_name] = model
        return cls(encoder=shared_encoder, taskmodels_dict=taskmodels_dict)

    @classmethod
    def get_encoder_attr_name(cls, model):
        """
        The encoder transformer is named differently in each model "architecture".
        This method lets us get the name of the encoder attribute
        """
        model_class_name = model.__class__.__name__
        if model_class_name.startswith("Bert"):
            return "bert"
        elif model_class_name.startswith("DistilBert"):
            return "distilbert"
        elif model_class_name.startswith("Span"):
            return "SpanBERT/spanbert-base-cased"
        else:
            raise KeyError(f"Add support for new model {model_class_name}")

    def forward(self, task_name, **kwargs):
        return self.taskmodels_dict[task_name](**kwargs)

In [10]:
## The BERTCRF architecture
log_soft = F.log_softmax

class BertCRF(BertPreTrainedModel):

    _keys_to_ignore_on_load_unexpected = [r"pooler"]

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel(config, add_pooling_layer=False)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.crf = CRF(num_tags=config.num_labels, batch_first=True)
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`):
            Labels for computing the token classification loss. Indices should be in ``[0, ..., config.num_labels -
            1]``.
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)

        
        loss = None
        if labels is not None:
            labels[labels==-100] = len(ner_label_names) ## CRF cannot handle -100 as index
            log_likelihood, tags = self.crf(logits, labels), self.crf.decode(logits)
            loss = 0 - log_likelihood
        else:
            tags = self.crf.decode(logits)
        tags = torch.Tensor(tags)

        if not return_dict:
            output = (tags,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output
        return loss, tags

In [11]:
model_name = "SpanBERT/spanbert-base-cased"
multitask_model = MultitaskModel.create(
    model_name=model_name,
    model_type_dict={
        "qa": transformers.AutoModelForQuestionAnswering,
        "ner": BertCRF,
    },
    model_config_dict={
        "qa": transformers.AutoConfig.from_pretrained(model_name),
        "ner": transformers.AutoConfig.from_pretrained(model_name, num_labels = len(ner_label_names)+1),
    },
)

Downloading:   0%|          | 0.00/413 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/215M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertCRF were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['classifier.weight', 'crf.end_transitions', 'crf.start_transitions', 'crf.transitions', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
if model_name.startswith("bert-"):
    print(multitask_model.encoder.embeddings.word_embeddings.weight.data_ptr())
    print(multitask_model.taskmodels_dict["qa"].bert.embeddings.word_embeddings.weight.data_ptr())
    print(multitask_model.taskmodels_dict["ner"].bert.embeddings.word_embeddings.weight.data_ptr())
else:
    print("Exercise for the reader: add a check for other model architectures =)")

Exercise for the reader: add a check for other model architectures =)


In [13]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [14]:
max_length = 128

ner_label_names = dataset_dict["ner"]["train"].features["ner_tags"].feature.names
# ner_label_names = ['O', 'B-QUOTE', 'I-QUOTE',]
def convert_to_ner_features(examples):
    tokenized_samples = tokenizer.batch_encode_plus(examples["tokens"], is_split_into_words=True, 
                                                    max_length=max_length, pad_to_max_length=True)
    #tokenized_samples is not a datasets object so this alone won't work with Trainer API, hence map is used 
    #so the new keys [input_ids, labels (after adjustment)]
    #can be added to the datasets dict for each train test validation split
    total_adjusted_labels = []
    print(len(tokenized_samples["input_ids"]))
    for k in range(0, len(tokenized_samples["input_ids"])):
        prev_wid = -1
        word_ids_list = tokenized_samples.word_ids(batch_index=k)
        existing_label_ids = examples["ner_tags"][k]
        i = -1
        adjusted_label_ids = []
    
        for wid in word_ids_list:
            if(wid is None):
                adjusted_label_ids.append(-100)
            elif(wid!=prev_wid):
                i = i + 1
                adjusted_label_ids.append(existing_label_ids[i])
                prev_wid = wid
            else:
                label_name = ner_label_names[existing_label_ids[i]]
                adjusted_label_ids.append(existing_label_ids[i])
            
        total_adjusted_labels.append(adjusted_label_ids)
    tokenized_samples["labels"] = total_adjusted_labels
    return tokenized_samples

max_length = 512 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
pad_on_right = tokenizer.padding_side == "right"



def convert_to_qa_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples



convert_func_dict = {
    "qa": convert_to_qa_features,
    "ner": convert_to_ner_features,
}

In [15]:
columns_dict = {
    "qa": ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    "ner": ['input_ids', 'attention_mask', 'labels'],
}

features_dict = {}
for task_name, dataset in dataset_dict.items():
    features_dict[task_name] = {}
    if task_name != 'qa':
        
        for phase, phase_dataset in dataset.items():
            features_dict[task_name][phase] = phase_dataset.map(
                convert_func_dict[task_name],
                batched=True,
                load_from_cache_file=False,
                
            )
            print(task_name, phase, len(phase_dataset), len(features_dict[task_name][phase]))
            features_dict[task_name][phase].set_format(
                type="torch", 
                columns=columns_dict[task_name],
            )
        
    else:
        for phase, phase_dataset in dataset.items():
            features_dict[task_name][phase] = phase_dataset.map(
                convert_func_dict[task_name],
                batched=True,
                load_from_cache_file=False,
                remove_columns=phase_dataset.column_names
            )

            features_dict[task_name][phase].set_format(
                type="torch", 
                columns=columns_dict[task_name],
            )

    print(task_name, phase, len(phase_dataset), len(features_dict[task_name][phase]))

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

qa validation 526 530


  0%|          | 0/1 [00:00<?, ?ba/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


420
ner train 420 420


  0%|          | 0/1 [00:00<?, ?ba/s]

106
ner validation 106 106
ner validation 106 106


## So far we have done
- create a multitask  model
- We have created a (cached) dictionary of featurized inputs for each of our tasks

In [16]:
import dataclasses
from torch.utils.data.dataloader import DataLoader
from transformers import DataCollator
from transformers.data.data_collator import InputDataClass
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data.sampler import RandomSampler
from typing import List, Union, Dict
from transformers import default_data_collator



print(type(default_data_collator))
class NLPDataCollator():
    """
    Extending the existing DataCollator to work with NLP dataset batches
    """
    def __call__(self, features: List[Union[InputDataClass, Dict]]) -> Dict[str, torch.Tensor]:
        first = features[0]
        if isinstance(first, dict) and "labels" in first:
            # NLP data sets current works presents features as lists of dictionary
            # (one per example), so we  will adapt the collate_batch logic for that
            if "labels" in first and first["labels"] is not None:
                if first["labels"].dtype == torch.int64:
                    labels = torch.tensor([f["labels"] for f in features], dtype=torch.long)
                else:
                    labels = torch.tensor([f["labels"] for f in features], dtype=torch.float)
                batch = {"labels": labels}
            for k, v in first.items():
                if k != "labels" and v is not None and not isinstance(v, str):
                    batch[k] = torch.stack([f[k] for f in features])
            return batch
        else:
          # otherwise, revert to using the default collate_batch
        #   return DefaultDataCollator().collate_batch(features)
            return default_data_collator

data_collator_ = default_data_collator
# data_collator_ = NLPDataCollator()

class StrIgnoreDevice(str):
    """
    This is a hack. The Trainer is going call .to(device) on every input
    value, but we need to pass in an additional `task_name` string.
    This prevents it from throwing an error
    """
    def to(self, device):
        return self


class DataLoaderWithTaskname:
    """
    Wrapper around a DataLoader to also yield a task name
    """
    def __init__(self, task_name, data_loader):
        self.task_name = task_name
        self.data_loader = data_loader

        self.batch_size = data_loader.batch_size
        self.dataset = data_loader.dataset

    def __len__(self):
        return len(self.data_loader)
    
    def __iter__(self):
        for batch in self.data_loader:
            batch["task_name"] = StrIgnoreDevice(self.task_name)
            yield batch


class MultitaskDataloader:
    """
    Data loader that combines and samples from multiple single-task
    data loaders.
    """
    def __init__(self, dataloader_dict):
        self.dataloader_dict = dataloader_dict
        self.num_batches_dict = {
            task_name: len(dataloader) 
            for task_name, dataloader in self.dataloader_dict.items()
        }
        self.task_name_list = list(self.dataloader_dict)
        self.dataset = [None] * sum(
            len(dataloader.dataset) 
            for dataloader in self.dataloader_dict.values()
        )

    def __len__(self):
        return sum(self.num_batches_dict.values())

    def __iter__(self):
        """
        For each batch, sample a task, and yield a batch from the respective
        task Dataloader.

        We use size-proportional sampling, but you could easily modify this
        to sample from some-other distribution.
        """
        task_choice_list = []
        for i, task_name in enumerate(self.task_name_list):
            task_choice_list += [i] * self.num_batches_dict[task_name]
        task_choice_list = np.array(task_choice_list)
        np.random.shuffle(task_choice_list)
        dataloader_iter_dict = {
            task_name: iter(dataloader) 
            for task_name, dataloader in self.dataloader_dict.items()
        }
        for task_choice in task_choice_list:
            task_name = self.task_name_list[task_choice]
            yield next(dataloader_iter_dict[task_name])    

class MultitaskTrainer(transformers.Trainer):

    def get_single_train_dataloader(self, task_name, train_dataset):
        """
        Create a single-task data loader that also yields task names
        """
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")
        train_sampler = (
            RandomSampler(train_dataset)
            if self.args.local_rank == -1
            else DistributedSampler(train_dataset)
        )

        data_loader = DataLoaderWithTaskname(
            task_name=task_name,
            data_loader=DataLoader(
              train_dataset,
              batch_size=self.args.train_batch_size,
              sampler=train_sampler,
            #   collate_fn=data_collator_,
            ),
        )

        # if is_tpu_available():
        #     data_loader = pl.ParallelLoader(
        #         data_loader, [self.args.device]
        #     ).per_device_loader(self.args.device)
        return data_loader

    def get_train_dataloader(self):
        """
        Returns a MultitaskDataloader, which is not actually a Dataloader
        but an iterable that returns a generator that samples from each 
        task Dataloader
        """
        return MultitaskDataloader({
            task_name: self.get_single_train_dataloader(task_name, task_dataset)
            for task_name, task_dataset in self.train_dataset.items()
        })

    

<class 'function'>


In [17]:
# import os
# os.environ["WANDB_DISABLED"] = "true"
# train_dataset = {
#     task_name: dataset["train"] 
#     for task_name, dataset in features_dict.items()
# }


# # eval_dataset = {
# #     task_name: dataset["validation"] 
# #     for task_name, dataset in features_dict.items()
# # }

# # train_dataset = features_dict['qa']['train']
# # eval_dataset = features_dict['qa']['validation']
# trainer = MultitaskTrainer(
#     model=multitask_model,
#     args=transformers.TrainingArguments(
#         output_dir="./models/multitask_model_bert_only",
#         overwrite_output_dir=True,
#         learning_rate=2e-5,
#         do_train=True,
#         num_train_epochs=20,
#         # Adjust batch size if this doesn't fit on the Colab GPU
#         per_device_train_batch_size=4,  
#         save_steps=3000,
#     ),
#     data_collator=data_collator_,
#     train_dataset=train_dataset,
#     # eval_dataset=eval_dataset,
# )


# I am starting fresh

In [30]:
train_dataset = {
    task_name: dataset["train"] 
    for task_name, dataset in features_dict.items()
}


eval_dataset = {
    task_name: dataset["validation"] 
    for task_name, dataset in features_dict.items()
}

In [43]:
ids, contexts, questions, answers = read_squad('../input/quote-recommendation/quote_to_squad_train.json')
count = int(ids[-1].split("_")[1])
print(count)
target = count + 20
for context, qa in context_quote_paragraph_gandhi.items():
    ans_dict = {'answer_start': [qa['start_index']], 'text': [str(qa['quote'])]}

    contexts.append(str(qa['paragraph']))
    questions.append(str(context))
    answers.append(ans_dict)
    ids.append("quoter_%s"%count)
    count+= 1
    if count>target:
        break
        
        
        
quote_dict_quoter = {
    "id": ids,
    "context": contexts,
    "question": questions,
    "answers": answers
}


ds_quoter = datasets.Dataset.from_dict(quote_dict_quoter)


with open('../input/quote-recommendation/overall_data_qa_formatted_new.json', 'rb') as content:
    context_quote_paragraph_gandhi = json.load(content)

# with open('/content/quotus_converted.json', 'rb') as test:
#     context_quote_test = json.load(test)


count = 1
ids, train_contexts, train_questions, train_answers = [], [], [], []

for context, qa in context_quote_paragraph_gandhi.items():
    contexts = []
    questions = []
    answers = []
    
    ans_dict = {'answer_start': [qa['start_index']], 'text': [str(qa['quote'])]}

    train_contexts.append(str(qa['paragraph']))
    train_questions.append(str(context))
    train_answers.append(ans_dict)
    ids.append("gandhi_%s"%count)
    count+=1 


quote_dict_gandhi = {
    "id": ids,
    "context": train_contexts,
    "question": train_questions,
    "answers": train_answers
}


ds_gandhi = datasets.Dataset.from_dict(quote_dict_gandhi)

# # 80% train, 20% test + validation
# train_testvalid = ds.train_test_split(0.2)
# # gather everyone if you want to have a single DatasetDict
# quote_dataset = datasets.DatasetDict({
#     'train': train_testvalid['train'],
#     'validation': train_testvalid['test']})

quote_dataset = datasets.DatasetDict({
    'train': ds_quoter,
    'validation': ds_gandhi})
quote_dataset

10253


DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 10274
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 526
    })
})

In [44]:
## Few shot fine tuning

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import os
os.environ["WANDB_DISABLED"] = "true"

train_dataset = {
    task_name: dataset["train"] 
    for task_name, dataset in features_dict.items()
}


eval_dataset = {
    task_name: dataset["validation"] 
    for task_name, dataset in features_dict.items()
}

# train_dataset = train_dataset['qa'][:].update(eval_dataset['qa'][:20])
# eval_dataset = eval_dataset['qa'][20:]

# model = transformers.AutoConfig.from_pretrained("./models/multitask_model/checkpoint-12000").to(device) ##HERE I LOAD FROM CHECKPOINT
trainer = MultitaskTrainer(
    model=multitask_model,
    args=transformers.TrainingArguments(
        output_dir="./models/multitask_model_fine_tuned",
        overwrite_output_dir=True,
        learning_rate=2e-5,
        do_train=True,
        num_train_epochs=2,
        # Adjust batch size if this doesn't fit on the Colab GPU
        per_device_train_batch_size=4,
        save_steps=3000,
        prediction_loss_only=True,
    ),
    data_collator=data_collator_,
    train_dataset=train_dataset,
    # eval_dataset=eval_dataset,
)
trainer.train("../input/checkpoint")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Loading model from ../input/checkpoint).
***** Running training *****
  Num examples = 10809
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 5406


Step,Training Loss
500,0.083600
1000,0.384300
1500,1.084300
2000,0.206300
2500,0.411700
3000,0.029600
3500,1.469900
4000,0.290800
4500,0.438400
5000,0.180300


Saving model checkpoint to ./models/multitask_model_fine_tuned/checkpoint-3000
Configuration saved in ./models/multitask_model_fine_tuned/checkpoint-3000/config.json
Model weights saved in ./models/multitask_model_fine_tuned/checkpoint-3000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=5406, training_loss=0.42440263130379924, metrics={'train_runtime': 1396.9597, 'train_samples_per_second': 0.003, 'train_steps_per_second': 3.87, 'total_flos': 5648919159656448.0, 'train_loss': 0.42440263130379924, 'epoch': 2.0})

In [ ]:
features_dict

In [ ]:
def save_model(multitask_model):
    for task_name in ["qa", "ner"]:
        multitask_model.taskmodels_dict[task_name].config.to_json_file(
            f"./{task_name}_model/config.json"
        )
        torch.save(
            multitask_model.taskmodels_dict[task_name].state_dict(),
            f"./{task_name}_model/pytorch_model.bin",
        )
        tokenizer.save_pretrained(f"./{task_name}_model/")

In [ ]:
model = torch.load('../input/checkpoint/pytorch_model.bin')

In [ ]:
#model

In [ ]:
def multitask_eval_fn(multitask_model, model_name, features_dict, batch_size=8):
    preds_dict = {}
    # tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
    for task_name in ["qa"]:
        val_len = len(features_dict[task_name]["validation"])
        acc = []
        """
        inputs = list(zip(example_batch['sentence1'], example_batch['sentence2']))
    features = tokenizer.batch_encode_plus(
        inputs, max_length=max_length, pad_to_max_length=True
    )
    features["labels"] = example_batch["label"]
        """
        for index in range(0, val_len, batch_size):

            # batch = list(zip(features_dict[task_name]["validation"][
            #     index : min(index + batch_size, val_len)
            # ]["sentence1"], 
            # features_dict[task_name]["validation"][
            #     index : min(index + batch_size, val_len)
            # ]["sentence2"]
            # ))
            batch = {}
            batch['input_ids'] = features_dict[task_name]["validation"][
                index : min(index + batch_size, val_len)
            ]['input_ids'].cuda()
            batch['attention_mask'] = features_dict[task_name]["validation"][
                index : min(index + batch_size, val_len)
            ]['attention_mask'].cuda()
            start_true = features_dict[task_name]["validation"][
                index : min(index + batch_size, val_len)
            ]['start_positions']
            end_true = features_dict[task_name]["validation"][
                index : min(index + batch_size, val_len)
            ]['end_positions']

            # labels = features_dict[task_name]["validation"][
            #     index : min(index + batch_size, val_len)
            # ]["labels"]
            # inputs = tokenizer(batch, max_length=512, padding=True)
            # inputs["input_ids"] = torch.LongTensor(inputs["input_ids"])
            # inputs["attention_mask"] = torch.LongTensor(inputs["attention_mask"])
            outputs = multitask_model("qa", **batch)
            # pull preds out
            start_pred = torch.argmax(
                torch.FloatTensor(torch.softmax(outputs['start_logits'], dim=1).detach().cpu().tolist()),
                dim=1,
            )
            end_pred = torch.argmax(
                torch.FloatTensor(torch.softmax(outputs['end_logits'], dim=1).detach().cpu().tolist()),
                dim=1,
            )
            # calculate accuracy for both and append to accuracy list
            acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
            acc.append(((end_pred == end_true).sum()/len(end_pred)).item())
            # acc += sum(np.array(predictions) == np.array(labels))
        acc = sum(acc)/len(acc)
        print(f"Task name: {task_name} \t Accuracy: {acc}")

In [ ]:
features_dict[task_name]["validation"][5:7]

In [ ]:
batch_size=8
# tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
for task_name in ["qa"]:
    val_len = len(features_dict[task_name]["validation"])
    acc = []
    for index in range(0, val_len, batch_size):
        batch = {}
        batch['input_ids'] = features_dict[task_name]["validation"][
            index : min(index + batch_size, val_len)
        ]['input_ids'].cuda()
        batch['attention_mask'] = features_dict[task_name]["validation"][
            index : min(index + batch_size, val_len)
        ]['attention_mask'].cuda()
        start_true = features_dict[task_name]["validation"][
            index : min(index + batch_size, val_len)
        ]['start_positions']
        end_true = features_dict[task_name]["validation"][
            index : min(index + batch_size, val_len)
        ]['end_positions']
        
        
        outputs = multitask_model("qa", **batch)
        start_pred = torch.argmax(outputs['start_logits'], dim=1).detach().cpu()
        end_pred = torch.argmax(outputs['end_logits'], dim=1).detach().cpu()
        # calculate accuracy for both and append to accuracy list
        acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
        acc.append(((end_pred == end_true).sum()/len(end_pred)).item())
        # acc += sum(np.array(predictions) == np.array(labels))
    acc = sum(acc)/len(acc)
    print(f"Task name: {task_name} \t Accuracy: {acc}")

In [45]:
def prepare_validation_features(examples): 
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [46]:
validation_features = dataset_dict['qa']["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=dataset_dict['qa']["validation"].column_names
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
features_dict['qa'][phase]

In [47]:
## getting raw probabilities as prediction
def get_preds_qa(multitask_model, model_name, validation_features):
    preds_dict = {}
    task_name = "qa"
    val_len = len(validation_features)
    start_logits = []
    end_logits = []
    for index in range(0, val_len):
        inputs = {}
        inputs['input_ids'] = torch.tensor(validation_features[index:index+1]['input_ids']).cuda()
        inputs['attention_mask'] = torch.tensor(validation_features[index:index+1]['attention_mask']).cuda()
        # start_true = validation_features[index]['start_positions']
        # end_true = validation_features[index]['end_positions']
        outputs = multitask_model("qa", **inputs)
        # print(outputs['start_logits'][0].detach().cpu().tolist())
        start_logits.append(outputs['start_logits'][0].detach().cpu().tolist())
        end_logits.append(outputs['end_logits'][0].detach().cpu().tolist())

    start_logits = np.array(start_logits)
    end_logits = np.array(end_logits)

    return start_logits, end_logits
raw_predictions = get_preds_qa(multitask_model, model_name, validation_features)

In [68]:
from tqdm.auto import tqdm
import numpy as np
import collections

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.defaultdict(list)

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        #if len(valid_answers) > 0:
        best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0:5]
        temp_list = []
        for ans in best_answer:
            temp_list.append(ans['text'])
       
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        predictions[example["id"]] = temp_list
        
    return predictions

final_predictions = postprocess_qa_predictions(dataset_dict['qa']["validation"], validation_features, raw_predictions)

Post-processing 526 example predictions split into 530 features.


  0%|          | 0/526 [00:00<?, ?it/s]

In [ ]:
len(quote_dataset['validation']['answers'])

zipped = zip(dictionary.iteritems(), num_list)

for (key, value), num_list_entry in zipped:
    print key
    print value
    print num_list_entry

In [117]:
len(quote_dataset['validation']["answers"])

526

In [125]:
zipped = zip(final_predictions.items(),quote_dataset['validation']['context'],quote_dataset['validation']["answers"])

# import pandas library as pd
import pandas as pd
  
# create an Empty DataFrame object
df = pd.DataFrame(columns=['index', 'context', 'ground_truth','predictions','is_relevant'])
index = []
gt = []
con = []
p = []
for (key, value), num_list_entry,ans in zipped:
    index.append(key)
    con.append(num_list_entry)
    gt.append(ans["text"])
    i = 0
    final_val = [""]*5
    for i,val in enumerate(value):
        final_val[i] = f"{(i+1)} --> {val}"
        
    p.append(final_val)
        
df['index'] = index
df['context'] = con
df['ground_truth'] = gt
df['predictions'] = p


In [126]:
df.head()

,index,context,ground_truth,predictions,is_relevant
0,gandhi_1,"I congratulate you, my dear fellow. In this wo...",[In this world there are only two tragedies. O...,[1 --> In this world there are only two traged...,NaN
1,gandhi_2,"With the abolition of private property, then, ...",[To live is the rarest thing in the world. Mos...,[1 --> To live is the rarest thing in the worl...,NaN
2,gandhi_3,When Jesus talks about the poor he simply mean...,[Ordinary riches can be stolen from a man. Rea...,[1 --> Ordinary riches can be stolen from a ma...,NaN
3,gandhi_4,The chief merit of Madame Bovary is not the mo...,[The critic has to educate the public; the art...,[1 --> The chief merit of Madame Bovary is not...,NaN
4,gandhi_5,"""Oh, anything,"" Wilde replied. ""Anything, no m...",[I have the simplest tastes. I am always satis...,"[1 --> I have the simplest tastes. I am, 2 -->...",NaN


In [127]:
df.to_csv("annotation_map_final.csv")

In [116]:
zipped = zip(final_predictions.items(),quote_dataset['validation']['answers'])
for (key, value), num_list_entry in zipped:
    print(f"{key} --> {num_list_entry['text'][0]} -------------------")
    for i,val in enumerate(value):
        print(f"predicted {i+1}  --> {val}")

gandhi_1 --> In this world there are only two tragedies. One is not getting what one wants, and the other is getting it. -------------------
predicted 1  --> In this world there are only two tragedies
predicted 2  --> In this
predicted 3  --> In this world there are only two
predicted 4  --> In this world there are
predicted 5  --> In this world there are only
gandhi_2 --> To live is the rarest thing in the world. Most people exist, that is all. -------------------
predicted 1  --> To live is the rarest thing in the world. Most people exist, that is all
predicted 2  --> To live is the rarest thing in the world
predicted 3  --> To live is the rarest thing
predicted 4  --> To live is the rarest thing in the world. Most people
predicted 5  --> To live is the rarest thing in the world. Most people exist
gandhi_3 --> Ordinary riches can be stolen from a man. Real riches cannot. In the treasury-house of your soul, there are infinitely precious things, that may not be taken from you. --------

In [ ]:
# final_predictions
metric = datasets.load_metric("squad_v2" if squad_v2 else "squad")

if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in dataset_dict['qa']["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

In [ ]:
features_dict[task_name]["validation"]['input_ids'][]

In [ ]:
features_dict[task_name]["validation"][1]['start_positions']

In [ ]:
answer_start = features_dict[task_name]["validation"][1]['start_positions']
answer_end = features_dict[task_name]["validation"][1]['end_positions']
print(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(features_dict[task_name]["validation"]['input_ids'][1][answer_start:answer_end])))

In [ ]:
quote_dataset['validation']

In [ ]:
len(quote_dataset['validation']['context'])

In [ ]:
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return "".join(text).lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))
def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [ ]:
quote_dataset['validation']['answers'][5]['text']

In [ ]:
(features_dict['qa']['validation'])

In [ ]:
#len(features_dict[task_name]["validation"])  -> 530
len(quote_dataset['validation']['answers']) #-> 526

In [ ]:
f1 = 0
batch_size=8
# tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
for task_name in ["qa"]:
    val_len = len(quote_dataset['validation']['answers'])
    for index in range(0, val_len, batch_size):
        #print(f"-------------------------- {index} ----------------------------------")
        batch = {}
        batch['input_ids'] = features_dict[task_name]["validation"][
            index : min(index + batch_size, val_len)
        ]['input_ids'].cuda()
        batch['attention_mask'] = features_dict[task_name]["validation"][
            index : min(index + batch_size, val_len)
        ]['attention_mask'].cuda()
        start_true = features_dict[task_name]["validation"][
            index : min(index + batch_size, val_len)
        ]['start_positions']
        end_true = features_dict[task_name]["validation"][
            index : min(index + batch_size, val_len)
        ]['end_positions']
        
        actual_answers = []
        for id in range(index,min(index+batch_size,val_len)):
            actual_answers .append(quote_dataset['validation']['answers'][id]['text'])
        
        outputs = multitask_model("qa", **batch)
    

        start_pred = torch.argmax(outputs['start_logits'], dim=1).detach().cpu()
        end_pred = torch.argmax(outputs['end_logits'], dim=1).detach().cpu()
        predicted_answers = []
        for i in range(len(start_pred)):
            predicted_answers.append(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(batch['input_ids'][i][start_pred[i]:end_pred[i]+1])))
        for actual_ans,pred_ans in zip(actual_answers,predicted_answers):
#             print("--------------------actual ans --------------------------")
#             print(actual_ans)
#             print("--------------------predicted answer ---------------------")
#             print(pred_ans)
            f1 += compute_f1(pred_ans,actual_ans)
        
    print(f1/val_len)

# top 5 answers 

In [55]:
f1 = 0
batch_size=8
# tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
for task_name in ["qa"]:
    val_len = len(quote_dataset['validation']['answers'])
    for index in range(0, val_len, batch_size):
        #print(f"-------------------------- {index} ----------------------------------")
        batch = {}
        batch['input_ids'] = features_dict[task_name]["validation"][
            index : min(index + batch_size, val_len)
        ]['input_ids'].cuda()
        batch['attention_mask'] = features_dict[task_name]["validation"][
            index : min(index + batch_size, val_len)
        ]['attention_mask'].cuda()
        start_true = features_dict[task_name]["validation"][
            index : min(index + batch_size, val_len)
        ]['start_positions']
        end_true = features_dict[task_name]["validation"][
            index : min(index + batch_size, val_len)
        ]['end_positions']
        
        actual_answers = []
        for id in range(index,min(index+batch_size,val_len)):
            actual_answers .append(quote_dataset['validation']['answers'][id]['text'])
        
        outputs = multitask_model("qa", **batch)
    
        start_pred = torch.argmax(outputs['start_logits'], dim=1).detach().cpu()
        end_pred = torch.argmax(outputs['end_logits'], dim=1).detach().cpu()
        start_preds_top_5 = torch.topk(outputs['start_logits'], 5,dim=1)[1].detach().cpu()
        end_preds_top_5 = torch.topk(outputs['end_logits'], 5,dim=1)[1].detach().cpu()
        predicted_answers = []
        for i in range(len(start_pred)):
            top_5_predicted_ans = []
            for j in range(5):
                top_5_predicted_ans.append(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(batch['input_ids'][i][start_preds_top_5[i][j]:end_preds_top_5[i][j]+1])))
   
            predicted_answers.append(top_5_predicted_ans)
    
        for actual_ans,pred_ans in zip(actual_answers,predicted_answers):
            print("--------------------actual ans --------------------------")
            print(actual_ans)
            print("--------------------predicted answers ---------------------")
            for i,p in enumerate(pred_ans):
                print(f"prediction {i+1}   -> {p}")            
            print("-----------------------------------------------------------------------------------")

--------------------actual ans --------------------------
['In this world there are only two tragedies. One is not getting what one wants, and the other is getting it.']
--------------------predicted answers ---------------------
prediction 1   -> [SEP]
prediction 2   -> [CLS] i wish you good luck, my friend. [SEP] i congratulate you, my dear fellow. in this world there are only two tragedies. one is not getting what one wants, and the other is getting it. the last is much the worst ; the last is a real tragedy! but i am interested to hear she does not love you. how long could you love a woman who didn't love you, cecil? [SEP]
prediction 3   -> 
prediction 4   -> i wish you good luck, my friend. [SEP] i congratulate you, my dear fellow. in this world there are only two tragedies
prediction 5   -> in this world
-----------------------------------------------------------------------------------
--------------------actual ans --------------------------
['To live is the rarest thing in the